# Imports

In [ ]:
import os
import anndata
import seaborn as sns
from sklearn import preprocessing

from Functions import get_folder_names, CalculateOutgrowth, transform_features, plate_bias_overview, save_adata

# User Input

In [ ]:
load_dir = "/Your/Absolute/Path"
save_dir = os.path.join(os.path.sep,*get_folder_names(load_dir)[:-1]) # Does not have to be changed if working with folder setup from Feature Extraction script
plot_save_dir = save_dir.replace("2_Results","3_Plots") # Does not have to be changed if working with folder setup from Feature Extraction script

# Set up plotting parameters
fig_param = {"axes.spines.right": False,
                  "axes.spines.top": False,
                  "axes.spines.left": True,
                  "axes.spines.bottom": True,
                  "pdf.fonttype": 42,
                  "font.size": 6,
                  "axes.labelsize": 6,
                  "axes.titlesize": 8,
                  "xtick.labelsize": 6,
                  "ytick.labelsize": 6,
                  "legend.fontsize": 6}

sns.set_theme(rc = fig_param, style = "ticks")

# Load & misc

In [ ]:
ad = anndata.read_h5ad(load_dir)

# Apply MinMax scaling and save into "minmax" layer
ad.layers["minmax"] = preprocessing.MinMaxScaler().fit_transform(ad.X)

# Calculate outgrowth
Calculates the outgrowth information per well by taking all indicated wells in the .xls setup file into account, even if no organoids are detected during the segmentation.

In [ ]:
ad = CalculateOutgrowth(ad = ad)

# log1p transformation of area-associated features
Perform log1p transformation on a subset of area-associated features in the AnnData object and visualize the results.

In [ ]:
ad = transform_features(ad = ad,
                   control_medium = "Ctrl", #Str used for control conditions in "Medium" tab of .xls setup file
                   plot_save_dir = plot_save_dir,
                   save_plot = True # Set to True to save plot
                  )

# Plate overview to check for positional bias
Generate heatmaps illustrating plate bias based on specified features and experimental conditions. Values are z-scores of well averages normalized within conditions.

In [ ]:
plate_bias_overview(plt_features = ["area", "circularity"], 
                    ad = ad,
                    plate_size = 384 # 384 or 96 plate setup
                   )

# Save AD

In [ ]:
save_adata(save_dir, "3_QC", ad)